In [32]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langsmith import utils, traceable


load_dotenv(dotenv_path='../.env')

utils.tracing_is_enabled()

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [33]:
from langchain_chroma.vectorstores import Chroma
from langchain_graph_retriever.transformers import ShreddingTransformer

#load the vector store "./data/recipe_qa_combined_chroma_db"
vector_store = Chroma(
    embedding_function=embeddings,
    collection_name="recipe_qa_combined",
    persist_directory="./data/recipe_qa_combined_chroma_db"
)

In [34]:
from graph_retriever.strategies import Eager
from langchain_graph_retriever import GraphRetriever

traversal_retriever = GraphRetriever(
    store = vector_store,
    edges = [("keywords", "keywords"), ("source_id", "source_id")],
    strategy = Eager(k=5, start_k=5, max_depth=3),
)

In [35]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)

@traceable(run_type="chain")
def format_docs(docs:list[Document], config=None):
    # formatted_docs = []
    # for doc in docs:
    #     doc_type = doc.metadata.get("type")
    #     f_doc = f"text: {doc.page_content} metadata: {doc.metadata}"
    #     if doc_type == "recipe":
    return {"context":"\n\n".join(
        f"text: {doc.page_content} metadata: {doc.metadata}" for doc in docs
    )}


# chain = (
#     {"sources": traversal_retriever}
#     | {"context": RunnableLambda(lambda x: format_docs(x['sources'])), "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

chain = (
    {"context": traversal_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    # | StrOutputParser()
)

In [37]:
response = chain.invoke("I'm in Ohio and I just had a small round chocolate that had peanut butter. I can't remeber the name of it. All I remember is that it had an 'eye' in the name. If you find it, get me the recipe")

In [38]:
print(response.content)

The treat you're referring to is likely "Buckeyes." Here are the details for the recipe:

### Buckeyes Recipe

#### Ingredients
- 1 stick butter, softened
- 1 lb. powdered sugar
- 2 c. crunchy peanut butter
- 3 c. Rice Krispies
- 12 oz. chocolate chips
- 1/3 stick paraffin wax

#### Directions
1. Mix butter, sugar, peanut butter, and Rice Krispies well.
2. Form mixture into balls about the size of a walnut.
3. Melt chocolate chips and paraffin in a double boiler.
4. Dip the balls in this chocolate mixture and place on wax paper until the chocolate is set. 

Enjoy making your Buckeyes!
